In [0]:
# %sh
raw_data_path = s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2018-01-04/platform=1/
unified_data_path_legacy = s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/
raw_domain_path = s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BASIC_METRICS/version=v1.0.0/
aws s3 ls  s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/

In [0]:


rank_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2020-09-30/"
spark.read.format("parquet").load(rank_path).createOrReplaceTempView("rank_category")
spark.sql("select * from rank_category where app_id = 1477487725 limit 10").show(10, False)
spark.sql("select distinct app_id, category_id from rank_category where app_id in (select app_id from {app_view}) and category_id = 100004".format(app_view=app_view)).show(100, False)


In [0]:

path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
spark.read.format("parquet").load(path).createOrReplaceTempView("test")
spark.sql("select last_updated, release_date,current_release_date, category_id  from test where id in  (1517006344)").show(10, False)


In [0]:

select * from ADL_USAGE_PAID.FACT_USAGE_CATEGORY_V1_CLUSTER_BY_DATE_COUNTRY_CODE
where product_type_code='app' 
and granularity_code='monthly' and date = '2020-09-30' 
and product_key = 20600010054776
and country_code = 'JP';

In [0]:

unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/granularity_code=monthly/date=2020-09-30/"
category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code=monthly/date=2020-09-30/"
# spark.read.format("delta").load(unified_path).createOrReplaceTempView("test_unified")
spark.read.format("delta").load(category_path).createOrReplaceTempView("test_category")
# spark.sql("select count(distinct product_key) as basic_count from test_unified ").show()
spark.sql("select count(1) as category_count from test_category where country_code <> 'ROW'").show()
# spark.sql("select count(1) as basic_count from test_unified ").show()
# spark.sql("select count(1) as category_null from test_category where category_key is null and unified_category_key is null").show()
# spark.sql("select * from test_category where product_key = 20600010054776 and country_code = 'JP'").show(10, False)
# spark.sql("select sum(est_active_users), sum(est_population) from test_unified where product_type_code='app' and granularity_code='monthly' and country_code <> 'ROW'").show()
# spark.sql("select count(distinct product_key, device_code, country_code, granularity_code, date) as basic_distinct_count from test_unified").show()

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType, ShortType

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)
date_count_sql =  """select  sum(count) from plproxy.execute_select($proxy$
SELECT count(1) as count
FROM mu.category_{granularity}_{device} where 
date = '{date}' and category_id = {legacy_category_id} $proxy$)
 t (count bigint) ;
"""


# granularity_list = ['daily', 'weekly', 'monthly']
granularity_list = ['weekly']
device_list = [1001, 1002, 2001, 2002]
ios_device_list = [ 2001, 2002]
gp_device_list = [1001, 1002]
device_code_list = ['android-phone', 'android-tablet', 'ios-phone', 'ios-tablet']
# device_code_list = ['ios-tablet']
device_mapping = {1001:'android-phone', 1002:'android-tablet', 2001:'ios-phone', 2002:'ios-tablet'}


legacy_ios_category_list = [6016, 6017, 6018, 6020, 6021, 6022, 6023, 6024, 36, 6001, 7001, 7002, 7003, 7004, 7005, 7006, 7007, 7008, 7009, 7010, 7011, 7012, 7013, 7014, 7015, 7016, 7017, 7018, 7019, 100, 360, 361, 362, 6000, 363, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015]

# legacy_ios_category_list = [6016, 6017, 6014]

ios_category_mapping = {6016: 800022, 6017: 800021, 6018: 800019, 6020: 800028, 6021: 800030, 6022: 800019, 6023: 800024, 6024: 800033, 36: 800000, 6001: 800038, 7001: 800002, 7002: 800003, 7003: 800002, 7004: 800004, 7005: 800006, 7006: 800006, 7007: 800006, 7008: 800007, 7009: 800008, 7010: 800008, 7011: 800010, 7012: 800005, 7013: 800011, 7014: 800012, 7015: 800013, 7016: 800014, 7017: 800012, 7018: 800015, 7019: 800016, 100: 800018, 360: 800041, 361: 800042, 362: 800043, 6000: 800020, 363: 800044, 6002: 800036, 6003: 800037, 6004: 800035, 6005: 800034, 6006: 800019, 6007: 800032, 6008: 800031, 6009: 800030, 6010: 800037, 6011: 800029, 6012: 800026, 6013: 800025, 6014: 800001, 6015: 800023}

legacy_gp_category_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75]

gp_category_mapping = {1: 800000, 2: 800001, 3: 800002, 4: 800005, 5: 800006, 6: 800005, 7: 800009, 8: 800011, 9: 800014, 10: 800017, 11: 800018, 12: 800019, 13: 800020, 14: 800019, 15: 800034, 16: 800021, 17: 800022, 18: 800023, 19: 800025, 20: 800019, 21: 800026, 22: 800027, 23: 800031, 24: 800028, 25: 800029, 26: 800030, 27: 800036, 28: 800031, 29: 800032, 30: 800033, 31: 800034, 32: 800035, 33: 800036, 34: 800037, 35: 800037, 36: 800038, 37: 800040, 38: 800002, 39: 800003, 40: 800016, 41: 800002, 42: 800004, 43: 800006, 44: 800006, 46: 800007, 47: 800008, 48: 800010, 49: 800005, 51: 800012, 52: 800013, 54: 800012, 55: 800015, 56: 800041, 57: 800042, 58: 800043, 59: 800044, 60: 800041, 61: 800041, 62: 800041, 63: 800041, 64: 800041, 65: 800041, 66: 800039, 67: 800032, 68: 800026, 69: 800033, 70: 800026, 71: 800026, 72: 800022, 73: 800024, 75: 800025}

legacy_to_individual_ios = {36: 100000, 6014: 100001, 7001: 100002, 7002: 100003, 7003: 100004, 7004: 100005, 7005: 100006, 7006: 100007, 7007: 100008, 7008: 100009, 7009: 100010, 7010: 100011, 7011: 100012, 7012: 100013, 7013: 100014, 7014: 100015, 7015: 100016, 7016: 100017, 7017: 100018, 7018: 100019, 7019: 100020, 100: 100021, 6018: 100022, 6000: 100023, 6022: 100024, 6017: 100025, 6016: 100026, 6015: 100027, 6023: 100028, 6013: 100029, 360: 100030, 361: 100031, 362: 100032, 363: 100033, 6012: 100034, 6021: 100035, 6020: 100064, 6011: 100065, 6010: 100066, 6009: 100067, 6008: 100068, 6007: 100069, 6006: 100070, 6024: 100071, 6005: 100072, 6004: 100073, 6003: 100075, 6002: 100076, 6001: 100077}

legacy_to_individual_gp = {1: 400000, 2: 400001, 38: 400002, 39: 400003, 41: 400004, 42: 400005, 43: 400006, 44: 400007, 6: 400008, 46: 400009, 47: 400010, 7: 400011, 48: 400012, 49: 400013, 8: 400014, 51: 400015, 52: 400016, 9: 400017, 54: 400018, 55: 400019, 10: 400020, 40: 400021, 3: 400022, 4: 400023, 5: 400024, 11: 400025, 66: 400026, 67: 400027, 69: 400028, 70: 400029, 12: 400030, 13: 400031, 14: 400032, 15: 400033, 71: 400034, 16: 400035, 17: 400036, 72: 400037, 18: 400038, 73: 400039, 19: 400040, 68: 400041, 20: 400042, 21: 400043, 22: 400044, 34: 400045, 24: 400046, 25: 400047, 26: 400048, 75: 400049, 27: 400050, 28: 400051, 29: 400052, 30: 400053, 31: 400054, 32: 400055, 33: 400056, 35: 400057, 23: 400058, 36: 400059, 37: 400060, 56: 400061, 60: 400062, 57: 400063, 58: 400064, 59: 400065, 61: 400066, 62: 400067, 63: 400068, 64: 400069, 65: 400070}


def get_plproxy_count(sql_str=date_count_sql):
    for device in gp_device_list:
        for granularity in granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity]:
                for category in legacy_gp_category_list:
                    result = query(PLPROXY_DSN, sql_str.format(granularity=granularity, date=date, device=device, legacy_category_id=category))
                    print device_mapping[device], granularity, date, category, result[0][0]

def get_unified_result():
    category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code={granularity}/date={date}/"
    for device in gp_device_list:
        for granularity in granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity]:
                for category in legacy_gp_category_list:
                    spark.read.format("delta").load(category_path.format(granularity=granularity, date=date)).createOrReplaceTempView("test_category")
                    df = spark.sql("select count(1) as count from test_category where device_code = '{device}' and category_key = {category} and country_code <> 'ROW'".format(device=device_mapping[device], category=legacy_to_individual_gp[category])).collect()
                    # print df[0][0]
                    print device_mapping[device], granularity, date, category, df[0][0]
                
def get_unified_tmp(date, device_code, individual_category_id):
    category_path_tmp = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code=monthly/date={date}/".format(date=date)
    spark.read.format("delta").load(category_path_tmp).createOrReplaceTempView("test_category_tmp")
    spark.sql("select distinct product_key as app_id from test_category_tmp where device_code = '{device_code}' and category_key = {individual_category_id} and country_code <> 'ROW'".format(device_code=device_code, individual_category_id=individual_category_id)).createOrReplaceTempView("unified_app")
    # spark.sql("select distinct country_code from test_category_tmp where device_code = 'ios-tablet'   and product_key = 284882215").show(100, False)
            
            
def get_plproxy_tmp(date, device_id, legacy_category_id):
    date_count_sql_tmp =  """select  distinct app_id from plproxy.execute_select($proxy$
SELECT distinct app_id as app_id
FROM mu.category_monthly_{device_id} where 
date = '{date}' and category_id in ({legacy_category_id}) $proxy$)
 t (app_id bigint);
""".format(device_id=device_id, date=date, legacy_category_id=legacy_category_id)
    print date_count_sql_tmp
    result = query(PLPROXY_DSN, date_count_sql_tmp)
    df_data = [Row(app_id=r[0]) for r in result]
    _schema =StructType([StructField("app_id", LongType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
    # print result[0][0]
    
def get_main_category(app_view):
    path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
    spark.read.format("parquet").load(path).createOrReplaceTempView("test")
    spark.sql("select id, last_updated, release_date,current_release_date, category_id  from test where id in  (select app_id from {app_view})".format(app_view=app_view)).show(100, False)
    
def get_rank_category(app_view):
    rank_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2020-09-30/"
    spark.read.format("parquet").load(rank_path).createOrReplaceTempView("rank_category")
    spark.sql("select * from rank_category where app_id = 1477487725 limit 10").show(10, False)
    spark.sql("select distinct app_id, category_id from rank_category where app_id in (select app_id from {app_view}) and category_id = 100004".format(app_view=app_view)).show(100, False)
    
def compare():
    date = '2020-09-30'
    device_id = 2001
    device_code = 'ios-phone'
    legacy_category_id = 6020
    individual_category_id = 100064
    df_plproxy = get_plproxy_tmp(date, device_id, legacy_category_id)
    get_unified_tmp(date, device_code, individual_category_id)
    df_plproxy.createOrReplaceTempView("plproxy_app")
    spark.sql("select app_id from plproxy_app except select app_id from unified_app").createOrReplaceTempView("plproxy_diff_unified")
    spark.sql("select app_id from unified_app except select app_id from plproxy_app ").createOrReplaceTempView("unifid_diff_plproxy")
    
    spark.sql("select app_id as plproxy_diff_unified from plproxy_diff_unified").show(100, False)
    spark.sql("select app_id as unifid_diff_plproxy from unifid_diff_plproxy").show(100, False)
    
    get_main_category("plproxy_diff_unified")
    get_main_category("unifid_diff_plproxy")
    
    
    
# get_rank_category("plproxy_diff_unified")
get_unified_result()

    

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


# begin_date = datetime(2019, 1, 1)
# end_date = datetime(2019, 1, 31)
begin_date = datetime(2013, 1, 1)
end_date = datetime(2013, 12, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

print DATE_GRANULARITY_MAPPINGLIST["monthly"]
print DATE_GRANULARITY_MAPPINGLIST["weekly"]
print DATE_GRANULARITY_MAPPINGLIST["daily"]

In [0]:

# est_average_active_days * est_active_users as est_total_active_days,
# est_average_bytes_per_session * est_active_users * est_average_session_count_per_user as est_total_bytes,
# est_active_users * est_average_session_count_per_user / est_share_of_session_product_in_main_category AS est_total_session_count_of_main_category,
# est_active_users * est_average_session_count_per_user * est_average_bytes_per_session / est_share_of_bytes_product_in_main_category AS est_total_bytes_of_main_category,
# est_active_users * est_average_time_per_user_milliseconds / est_share_of_time_product_in_main_category AS est_total_time_milliseconds_of_main_category,
# est_average_bytes_per_session * est_active_users * est_average_session_count_per_user * est_percentage_of_bytes_wifi_in_total * 1024 * 1024 * 1024 * 1024 AS est_wifi_bytes

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
usage_basic_before_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/"
granularity_list = ['monthly']
def test_bytes():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_after = spark.sql("""select count(1) from after_trans
            where (est_total_bytes = 0 and est_average_bytes_per_session <> 0 and est_average_session_count_per_user <> 0)
            or (est_total_session_count_of_main_category = 0 and est_share_of_session_product_in_main_category <> 0) 
            or (est_total_bytes_of_main_category = 0 and est_share_of_bytes_product_in_main_category <> 0)
            or (est_total_time_milliseconds_of_main_category = 0 and est_share_of_time_product_in_main_category <> 0)
            or (est_wifi_bytes = 0 and est_percentage_of_bytes_wifi_in_total <> 0 and est_average_session_count_per_user <> 0 and est_average_bytes_per_session <> 0)""").collect()
            if df_after[0][0] != 0:
                print granularity, date, "count", df_after[0][0]
            else:
                print granularity, date, "PASS"
                
def test_bytes_tmp():
    # monthly 2019-06-30 count 9646
    # monthly 2019-05-31 count 9061
    filter_str_after = "date between '2019-01-31' and 2019-12-31'' and granularity_code = 'monthly' and product_key = 20600000002550 and device_code = 'android-phone' and country_code = 'US'"
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
    # spark.sql("select count(1) from after_trans where est_total_bytes = 0 and est_average_bytes_per_session <> 0 and est_average_session_count_per_user <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_total_session_count_of_main_category = 0 and est_share_of_session_product_in_main_category <> 0").show(10, False)
    spark.sql("select est_total_bytes_of_main_category, est_share_of_bytes_product_in_main_category, est_average_bytes_per_session, est_average_session_count_per_user from after_trans").show(10, False)
    # spark.sql("select count(1) from after_trans where est_total_time_milliseconds_of_main_category = 0 and est_share_of_time_product_in_main_category <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_wifi_bytes = 0 and est_percentage_of_bytes_wifi_in_total <> 0 and est_average_session_count_per_user <> 0 and est_average_bytes_per_session <> 0").show(10, False)
    
    
def test_other_metrics():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            s = """select count(1) from after_trans
            where (est_install_penetration = 0  and est_open_rate <> 0)
            or (est_install_base = 0 and est_open_rate <> 0)
            or (est_population = 0 and est_usage_penetration <> 0)
            or (est_total_time_milliseconds = 0  and est_average_session_count_per_user <> 0)"""
            df_after = spark.sql(s).collect()
            if df_after[0][0] != 0:
                print granularity, date, "count", df_after[0][0]
            else:
                print granularity, date, "PASS"

def test_other_metrics_tmp():
    filter_str_after = "date = '2019-08-31' and granularity_code = 'monthly'"
    filter_str = "date ='2019-08-31' and granularity = 'monthly'"
    spark.read.format("delta").load(usage_basic_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
    # spark.sql("select count(1) from after_trans where est_install_penetration = 0  and est_open_rate <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_install_base = 0 and est_open_rate <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_population = 0 and est_usage_penetration <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_total_time_milliseconds = 0  and est_average_session_count_per_user <> 0").show(10, False)
    spark.sql("select count(1) from before_trans where est_total_time = 0 and est_average_session_per_user <> 0").show(10, False)
    spark.sql("select app_id, date, granularity, country_code, est_total_time, est_average_session_per_user, est_average_time_per_user, est_average_session_duration from before_trans where est_total_time = 0 and est_average_session_per_user <> 0 order by app_id desc limit 10").show(10, False)
    # spark.sql("select count(1) from before_trans").show(1, False)

test_bytes_tmp()



In [0]:

usage_raw_path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=MONTH/date=2020-08-31/"
unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
spark.read.format("delta").load(unified_path).createOrReplaceTempView("test_raw")
spark.sql("select * from test_raw limit 1").show(10, False)


In [0]:

granularity_list = ["monthly", "weekly", "daily"]
granularity_mapping = {"weekly":"WEEK", "daily":"DAY", "monthly":"MONTH"}
granularity_list = ["monthly"]

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
usage_basic_before_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/"
domain_before_transform = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity={granularity}/month={month}/date={date}/"
app_basic_raw_path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type={range_type}/date={date}/"

def compare_count():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str = "date ='{date}' and granularity = '{granularity}' and est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0  and device_code <> 'android-all' ".format(date=date, granularity=granularity)
            # filter_raw = "AU is not null and AU <> 0 and UP is not null and UP <> 0 ".format(date=date, granularity=granularity[:-2].upper())
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            # spark.read.format("parquet").load(app_basic_raw_path.format(range_type=granularity_mapping[granularity], date=date)).filter(filter_raw).createOrReplaceTempView("before_trans")
            spark.read.format("delta").load(usage_basic_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_before = spark.sql("select sum(est_average_active_users) as sum_au, count(1) as before_trans from before_trans ").collect()
            df_after = spark.sql("select  sum(est_active_users) as sum_au, count(1) as after_trans from after_trans").collect()
            diff =  abs(df_before[0][0] - ( df_after[0][0] if df_after[0][0] else 0 )) / df_before[0][0]
            if (df_before[0][1] == df_after[0][1]) and  diff < 0.000001:
                print granularity, date, df_before[0][1], df_after[0][1], df_before[0][0], df_after[0][0], "PASS"
            else:
                print granularity, date, df_before[0][1], df_after[0][1], df_before[0][0], df_after[0][0], "Failed"

def compare_sum_value():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str = "date ='{date}' and granularity = '{granularity}' and est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0 ".format(date=date, granularity=granularity)
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(usage_basic_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_before = spark.sql("select sum(est_average_active_users) as before_trans from before_trans ").collect()
            df_after = spark.sql("select sum(est_active_users) as after_trans from after_trans ").collect()
            diff =  abs(df_before[0][0] - df_after[0][0]) / df_before[0][0]
            if diff > 0.000001:
                print granularity, date, "before", df_before[0][0], "after", df_after[0][0]
            else:
                print granularity, date, "PASS"

def compare_domain_count():
    usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=domain/"
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str = "est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0 "
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(domain_before_transform.format(month=date[:-3].replace("-", ""), granularity=granularity[0], date=date)).filter(filter_str).createOrReplaceTempView("before_trans")
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_before = spark.sql("select count(1) as before_trans from before_trans ").collect()
            df_after = spark.sql("select count(1) as after_trans from after_trans").collect()
            if df_before[0][0] != df_after[0][0]:
                print granularity, date, "Failed", df_before[0][0], df_after[0][0]
            else:
                print granularity, date, "PASS", df_before[0][0], df_after[0][0]


def compare_domain_sum_value():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str = "est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0 "
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(domain_before_transform.format(month=date[:-3].replace("-", ""), granularity=granularity[0], date=date)).filter(filter_str).createOrReplaceTempView("before_trans")
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_before = spark.sql("select sum(est_average_active_users) as before_trans from before_trans").collect()
            df_after = spark.sql("select sum(est_active_users) as after_trans from after_trans").collect()
            
            diff =  abs(df_before[0][0] - df_after[0][0]) / df_before[0][0]
            if diff > 0.000001:
                print granularity, date, "Failed", df_before[0][0], df_after[0][0]
            else:
                print granularity, date, "PASS", df_before[0][0], df_after[0][0]
        
compare_count()




In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/

In [0]:

spark.sql("select count(1) as before_trans from before_trans where device_code <> 'android-all'").show(10, False)
spark.sql("select count(1) as after_trans from after_trans").show(10, False)

In [0]:

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=domain/"
domain_before_transform = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=m/month=201901/date=2019-01-31/"
filter_str = "est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0 "
filter_str_after = "date = '2019-01-31' and granularity_code = 'monthly'"
spark.read.format("parquet").load(domain_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
spark.sql("select count(distinct domain_id) as before_trans from before_trans ").show(10, False)
spark.sql("select count(distinct product_key) as after_trans from after_trans").show(10, False)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/

In [0]:

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
usage_basic_before_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/"
filter_str = "date ='2015-12-31' and granularity_code = 'monthly' "
# filter_before_str = "date ='2015-12-31' and granularity = 'monthly' and est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0 and est_open_rate <> 0 and est_open_rate is not null "

filter_before_str = "date ='2015-12-31' and granularity = 'monthly' and (est_open_rate = 0 or est_open_rate is null) and  est_average_active_users is not null and est_average_active_users <> 0 and est_usage_penetration is not null and est_usage_penetration <> 0"

spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str).createOrReplaceTempView("after_trans")
spark.read.format("delta").load(usage_basic_before_transform).filter(filter_before_str).createOrReplaceTempView("before_trans")
spark.sql("select app_id, device_code, est_average_active_users, est_usage_penetration, est_install_base, est_open_rate, est_installs, est_install_penetration from before_trans where country_code = 'US' and device_code = 'ios-phone' and est_installs <> 0 limit 100 ").show(100, False)
# spark.sql("select device_code, count(1) from after_trans group by device_code order by device_code desc").show(100, False)

# spark.sql("select distinct app_id from before_trans where device_code = 'android-phone' and country_code = 'US'").createOrReplaceTempView("before_app")
# spark.sql("select distinct product_key as app_id from after_trans where device_code = 'android-phone' and country_code = 'US'").createOrReplaceTempView("after_app")

# spark.sql("select app_id as before_diff_after from before_app except select app_id from after_app").show(1000, False)
# spark.sql("select count(1) from after_trans").show(100, False)
# spark.sql("select count(1) from after_trans where est_open_rate is null or est_open_rate = 0").show(10, False)


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141 -p 5432 -Ucitus_bdp_prod_app_int_qa -d aa_store_db << EOF
set search_path='usage';

select  est_share_of_category_bytes from usage_basic_kpi_fact_v1 where granularity ='weekly' 
and date = '2019-01-19' 
and country_code = 'US'
and device_code in ('android-phone')
and app_id = 20600000234348;

In [0]:
est_share_of_category_session,est_share_of_category_bytes,est_percent_of_wifi_total
select  max(date), min(date) from usage_basic_kpi_fact_v1 where est_share_of_category_session is not null and est_share_of_category_session <> 0 and granularity = 'daily';